# Exploratory Data Analysis (EDA)

## Introduction

In this notebook, we address the stakeholder questions using SQL queries on the cleaned streaming viewership data. The SQLite database is dynamically created from the cleaned CSV files.

---
## Step 1: Set Up the Environment

In [8]:

# Import necessary libraries
import pandas as pd
import sqlite3
import os

# Define the base project directory (update this to your project path)
base_project_path = '/Users/joshuastewart/Documents/Streaming Viewership Analysis'

# Define subdirectory paths
data_path = os.path.join(base_project_path, 'Data')
clean_data_path = os.path.join(data_path, 'cleaned_data')
database_file = os.path.join(data_path, 'streaming_viewership.db')

# Define file paths
users_file = os.path.join(clean_data_path, 'users_table.csv')
sessions_file = os.path.join(clean_data_path, 'sessions_table.csv')
videos_file = os.path.join(clean_data_path, 'videos_table.csv')
devices_file = os.path.join(clean_data_path, 'devices_table.csv')
locations_file = os.path.join(clean_data_path, 'locations_table.csv')
        

---
## Step 2: Load Cleaned Data and Create SQLite Database

In [11]:

# Load cleaned data into Pandas DataFrames
users = pd.read_csv(users_file)
sessions = pd.read_csv(sessions_file)
videos = pd.read_csv(videos_file)
devices = pd.read_csv(devices_file)
locations = pd.read_csv(locations_file)

# Create a SQLite database on disk
conn = sqlite3.connect(database_file)  # Saves database to 'streaming_viewership.db'

# Write DataFrames to SQLite
users.to_sql('Users', conn, index=False, if_exists='replace')
sessions.to_sql('Sessions', conn, index=False, if_exists='replace')
videos.to_sql('Videos', conn, index=False, if_exists='replace')
devices.to_sql('Devices', conn, index=False, if_exists='replace')
locations.to_sql('Locations', conn, index=False, if_exists='replace')

# Display confirmation of tables in the database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the SQLite database:")
print(tables)
        

Tables in the SQLite database:
        name
0      Users
1   Sessions
2     Videos
3    Devices
4  Locations


---
## Step 3: Exploratory Data Analysis (EDA)

### Content Preferences

**What are the top-performing genres among different user demographics (age groups, regions)?**
- To identify the top genres by demographic, we focused on the top three countries with the most users, segmented users into age groups, and calculated total viewing duration for each genre. The table shows the most-watched genre for each age group in these countries (Congo, Korea, and Wallis and Futuna), along with the total viewing duration.

In [53]:
query = '''
WITH TopCountries AS (
    SELECT 
        Country,
        COUNT(User_ID) AS Total_Users
    FROM Users
    GROUP BY Country
    ORDER BY Total_Users DESC
    LIMIT 3
),
RankedGenres AS (
    SELECT 
        CASE
            WHEN u.Age BETWEEN 0 AND 17 THEN '0-17'
            WHEN u.Age BETWEEN 18 AND 24 THEN '18-24'
            WHEN u.Age BETWEEN 25 AND 34 THEN '25-34'
            WHEN u.Age BETWEEN 35 AND 44 THEN '35-44'
            WHEN u.Age BETWEEN 45 AND 54 THEN '45-54'
            WHEN u.Age BETWEEN 55 AND 64 THEN '55-64'
            ELSE '65+'
        END AS Age_Group,
        u.Country,
        v.Genre,
        SUM(s."Duration_Watched (minutes)") AS Total_Viewing_Duration,
        COUNT(DISTINCT s.User_ID) AS Total_Users,
        ROW_NUMBER() OVER (
            PARTITION BY u.Country, 
                CASE
                    WHEN u.Age BETWEEN 0 AND 17 THEN '0-17'
                    WHEN u.Age BETWEEN 18 AND 24 THEN '18-24'
                    WHEN u.Age BETWEEN 25 AND 34 THEN '25-34'
                    WHEN u.Age BETWEEN 35 AND 44 THEN '35-44'
                    WHEN u.Age BETWEEN 45 AND 54 THEN '45-54'
                    WHEN u.Age BETWEEN 55 AND 64 THEN '55-64'
                    ELSE '65+'
                END
            ORDER BY SUM(s."Duration_Watched (minutes)") DESC
        ) AS Rank
    FROM Sessions s
    JOIN Users u ON s.User_ID = u.User_ID
    JOIN Videos v ON s.Video_ID = v.Video_ID
    WHERE u.Country IN (SELECT Country FROM TopCountries)
    GROUP BY 
        CASE
            WHEN u.Age BETWEEN 0 AND 17 THEN '0-17'
            WHEN u.Age BETWEEN 18 AND 24 THEN '18-24'
            WHEN u.Age BETWEEN 25 AND 34 THEN '25-34'
            WHEN u.Age BETWEEN 35 AND 44 THEN '35-44'
            WHEN u.Age BETWEEN 45 AND 54 THEN '45-54'
            WHEN u.Age BETWEEN 55 AND 64 THEN '55-64'
            ELSE '65+'
        END, 
        u.Country, 
        v.Genre
)
SELECT 
    Country,
    Age_Group as "Age Group",
    Genre,
    ROUND(Total_Viewing_Duration/60,2) as "Total Hours Watched (hrs)",
    Total_Users as "Total Users"
FROM RankedGenres
WHERE Rank = 1
ORDER BY Country, Age_Group;
'''
top_genre_with_users = pd.read_sql(query, conn)
top_genre_with_users

,Country,Age Group,Genre,Total Hours Watched (hrs),Total Users
0,Congo,0-17,Comedy,34.99,6
1,Congo,18-24,Documentary,43.27,7
2,Congo,25-34,Documentary,52.46,11
3,Congo,35-44,Documentary,56.06,12
4,Congo,45-54,Drama,39.74,8
5,Congo,55-64,Documentary,10.81,2
6,Congo,65+,Documentary,18.04,3
7,Korea,0-17,Drama,24.78,5
8,Korea,18-24,Thriller,36.25,7
9,Korea,25-34,Comedy,18.86,4


**Which genres drive the longest viewing durations, and how does this vary by subscription level?**
- To determine which genres drive the longest viewing durations and how this varies by subscription level, we calculated the average viewing duration for each genre across free and premium users. The table shows the genres ranked by their average viewing duration, separated by subscription status.

In [41]:

query = '''
SELECT 
    v.Genre,
    u.Subscription_Status as "Subscription Status",
    ROUND(AVG(s."Duration_Watched (minutes)"/60),2) as "Average Duration (hrs)"
FROM Sessions s
JOIN Users u ON s.User_ID = u.User_ID
JOIN Videos v ON s.Video_ID = v.Video_ID
GROUP BY v.Genre, u.Subscription_Status
ORDER BY "Average Duration (hrs)" DESC;
'''
genres_by_subscription = pd.read_sql(query, conn)
genres_by_subscription
        

,Genre,Subscription Status,Average Duration (hrs)
0,Action,Free,1.01
1,Action,Premium,1.01
2,Comedy,Premium,1.01
3,Documentary,Premium,1.01
4,Drama,Premium,1.01
5,Sci-Fi,Premium,1.01
6,Thriller,Premium,1.01
7,Comedy,Free,1.00
8,Documentary,Free,1.00
9,Drama,Free,1.00


### User Engagement

**How does playback quality (e.g., HD, 4K) impact session duration and user interactions?**
- To analyze how playback quality impacts session duration and user interactions, we calculated the average session duration and interaction events for each playback quality (SD, HD, 4K). Interaction events include user actions like clicks, likes, and shares during a session.

- The table shows SD playback leads to the highest average duration, while HD results in the most user interactions.

In [51]:

query = '''
SELECT 
    s.Playback_Quality,
    ROUND(AVG(s."Duration_Watched (minutes)"/60),2) as "Average Duration (hrs)",
   ROUND(AVG(s.Interaction_Events),2) AS "Average Interactions"
FROM Sessions s
GROUP BY s.Playback_Quality
ORDER BY "Average Duration (hrs)" DESC, "Average Interactions" DESC;
'''
playback_quality_impact = pd.read_sql(query, conn)
playback_quality_impact
        

,Playback_Quality,Average Duration (hrs),Average Interactions
0,SD,1.03,49.99
1,HD,0.99,52.15
2,4K,0.99,50.20


**Which devices are most commonly associated with higher engagement, and how do these trends differ between Premium and non-Premium users?**
- To identify devices associated with higher engagement, we calculated the average session duration and interaction events for each device type, segmented by subscription status (Premium and Free). The table shows that desktops have the highest engagement for Premium users, while laptops and tablets perform well among Free users.

In [57]:

query = '''
SELECT 
    u.Subscription_Status as "Subscription Status",
    d.Device_Type as "Device Type",
    ROUND(AVG(s."Duration_Watched (minutes)"),2) AS "Average Duration (hrs)",
    ROUND(AVG(s.Interaction_Events),2) AS "Average Interactions"
FROM Sessions s
JOIN Users u ON s.User_ID = u.User_ID
JOIN Devices d ON s.Device_ID = d.Device_ID
GROUP BY u.Subscription_Status, d.Device_Type
ORDER BY "Average Duration (hrs)" DESC, "Average Interactions" DESC;
'''
device_engagement = pd.read_sql(query, conn)
device_engagement
        

,Subscription Status,Device Type,Average Duration (hrs),Average Interactions
0,Premium,Desktop,60.76,50.49
1,Free,Laptop,60.67,50.33
2,Free,Tablet,60.66,49.97
3,Premium,Laptop,60.58,51.18
4,Premium,Tablet,60.51,51.39
5,Free,Smartphone,60.37,51.12
6,Premium,Smart TV,60.31,50.90
7,Free,Smart TV,60.31,50.89
8,Premium,Smartphone,60.20,50.21
9,Free,Desktop,59.89,50.39


### Customer Retention

**What factors (e.g., genre, device type, subscription level) correlate with higher session ratings?**
- To identify factors correlating with higher session ratings, we grouped ages into broader ranges and analyzed ratings and session durations by age group, genre, device type, and subscription status. The table shows high engagement from Premium users, with Action and Sci-Fi genres performing strongly across laptops and desktops for specific age groups.

In [65]:
query = '''
SELECT 
    u.Subscription_Status as "Subscription Status",
    CASE
        WHEN u.Age BETWEEN 0 AND 17 THEN '0-17'
        WHEN u.Age BETWEEN 18 AND 24 THEN '18-24'
        WHEN u.Age BETWEEN 25 AND 34 THEN '25-34'
        WHEN u.Age BETWEEN 35 AND 44 THEN '35-44'
        WHEN u.Age BETWEEN 45 AND 54 THEN '45-54'
        WHEN u.Age BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65+'
    END AS "Age Group",
    v.Genre,
    d.Device_Type as "Device Type",
    ROUND(AVG(s.Ratings),2) AS "Average Rating",
    ROUND(AVG(s."Duration_Watched (minutes)"/60),2) AS "Average Duration (hrs)"
FROM Sessions s
JOIN Users u ON s.User_ID = u.User_ID
JOIN Videos v ON s.Video_ID = v.Video_ID
JOIN Devices d ON s.Device_ID = d.Device_ID
GROUP BY "Age Group", v.Genre, "Device Type", "Subscription Status"
ORDER BY "Average Rating" DESC, "Average Duration (hrs)" DESC
LIMIT 10;
'''
factors_ratings = pd.read_sql(query, conn)
factors_ratings

,Subscription Status,Age Group,Genre,Device Type,Average Rating,Average Duration (hrs)
0,Premium,25-34,Action,Desktop,3.17,1.03
1,Premium,25-34,Action,Laptop,3.15,1.03
2,Premium,25-34,Action,Smartphone,3.15,1.01
3,Premium,25-34,Sci-Fi,Desktop,3.14,1.03
4,Free,65+,Comedy,Smartphone,3.14,1.00
5,Free,65+,Comedy,Desktop,3.14,0.98
6,Free,65+,Comedy,Laptop,3.14,0.97
7,Free,65+,Sci-Fi,Desktop,3.14,0.97
8,Free,65+,Sci-Fi,Laptop,3.14,0.97
9,Premium,25-34,Comedy,Desktop,3.13,1.03


### Platform Optimization

**Are there device types or playback qualities we should optimize further to enhance the viewing experience?**
- To identify device types or playback qualities to optimize, we calculated the average session duration and interaction events for each combination of device type and playback quality. Devices with SD playback, particularly laptops and tablets, should be prioritized for optimizing session duration. Meanwhile, HD playback on smartphones and Smart TVs leads to the highest user interactions, suggesting these combinations could benefit from enhancements to further improve engagement.

In [67]:

query = '''
SELECT 
    d.Device_Type as "Device Type",
    s.Playback_Quality as "Playback Quality",
    ROUND(AVG(s."Duration_Watched (minutes)"/60),2) as "Average Duration (hrs)",
    ROUND(AVG(s.Interaction_Events),2) as "Average Interactions"
FROM Sessions s
JOIN Devices d ON s.Device_ID = d.Device_ID
GROUP BY "Device Type", "Playback Quality"
ORDER BY "Average Duration (hrs)" DESC, "Average Interactions" DESC;
'''
device_quality_impact = pd.read_sql(query, conn)
device_quality_impact
        

,Device Type,Playback Quality,Average Duration (hrs),Average Interactions
0,Laptop,SD,1.04,49.57
1,Tablet,SD,1.03,50.13
2,Smartphone,SD,1.03,49.54
3,Desktop,SD,1.02,49.88
4,Smart TV,SD,1.02,49.71
5,Laptop,HD,1.00,52.31
6,Smart TV,HD,1.00,52.27
7,Tablet,HD,1.00,51.86
8,Smartphone,4K,1.00,50.17
9,Tablet,4K,1.00,50.04


**What steps can we take to improve engagement on underperforming subscription tiers?**
- To analyze engagement across subscription tiers, we calculated viewer counts, average session duration, interaction events, and ratings for Free and Premium users, along with minimum and maximum values for session duration and interactions. The table reveals that Free and Premium tiers have comparable engagement metrics, with Free users slightly outperforming Premium in ratings and interaction events. To improve Premium engagement, strategies could include exclusive content offerings, tailored interactive features, and enhancing playback quality to better justify the premium cost.

In [71]:
query = '''
SELECT 
    u.Subscription_Status as "Subscription Status",
    COUNT(DISTINCT s.User_ID) AS "Viewer Count",
    ROUND(AVG(s."Duration_Watched (minutes)"/60),2) as "Average Duration (hrs)",
    ROUND(MIN(s."Duration_Watched (minutes)"/60),2) as "Minimum Duration (hrs)",
    ROUND(MAX(s."Duration_Watched (minutes)"/60),2) as "Maximum Duration (hrs)",
    ROUND(AVG(s.Interaction_Events),2) AS "Average Interactions",
    ROUND(MIN(s.Interaction_Events),2) AS "Minimum Interactions",
    ROUND(MAX(s.Interaction_Events),2) AS "Maximum Interactions",
    ROUND(AVG(s.Ratings),2) AS "Average Rating"
FROM Sessions s
JOIN Users u ON s.User_ID = u.User_ID
GROUP BY "Subscription Status"
ORDER BY "Average Duration (hrs)" ASC;
'''
subscription_engagement = pd.read_sql(query, conn)
subscription_engagement

,Subscription Status,Viewer Count,Average Duration (hrs),Minimum Duration (hrs),Maximum Duration (hrs),Average Interactions,Minimum Interactions,Maximum Interactions,Average Rating
0,Free,3111,1.00,0.0,2.0,50.69,0.0,100.0,3.00
1,Premium,3103,1.01,0.0,2.0,50.87,0.0,100.0,2.98
